## Text Summerization

Steps:

1. Install the libraries
2. Load the data
3. Load the BERT pretained tokenizer model
4. Process the data and generate the encodings
5. Train the summerization model
6. Evaluate the model
7. Save the model
8. Summarize the text

### 1. Install Libraries

In [ ]:
!pip install transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.8 MB/s eta 0:00:00


### 2. Load Dataset

Start by loading the smaller California state bill subset of the **BillSum dataset** from the Hugging Face Datasets library.

In [ ]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test[:10%]")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
### Split Dataset
billsum = billsum.train_test_split(test_size=0.2)

In [ ]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nThis act shall be known, and may be cited, as the Third Validating Act of 2015.\nSEC. 2.\nAs used in this act:\n(a) “Public body” means all of the following:\n(1) The state and all departments, agencies, boards, commissions, and authorities of the state. Except as provided in paragraph (2), “public body” also means all cities, counties, cities and counties, districts, authorities, agencies, boards, commissions, and other entities, whether created by a general statute or a special act, including, but not limited to, the following:\nAgencies, boards, commissions, or entities constituted or provided for under or pursuant to the Joint Exercise of Powers Act (Chapter 5 (commencing with Section 6500) of Division 7 of Title 1 of the Government Code).\nAir pollution control districts of any kind.\nAir quality management districts.\nAirport districts.\nAssessment districts, benefit assessment districts, and sp

### 3. Load Tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

### 4. Process the Data

Prefix the input with a prompt so T5 knows this is a summarization task.

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], padding="max_length", truncation=True, max_length=128)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [ ]:
tokenized_billsum['train'][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nThis act shall be known, and may be cited, as the Third Validating Act of 2015.\nSEC. 2.\nAs used in this act:\n(a) “Public body” means all of the following:\n(1) The state and all departments, agencies, boards, commissions, and authorities of the state. Except as provided in paragraph (2), “public body” also means all cities, counties, cities and counties, districts, authorities, agencies, boards, commissions, and other entities, whether created by a general statute or a special act, including, but not limited to, the following:\nAgencies, boards, commissions, or entities constituted or provided for under or pursuant to the Joint Exercise of Powers Act (Chapter 5 (commencing with Section 6500) of Division 7 of Title 1 of the Government Code).\nAir pollution control districts of any kind.\nAir quality management districts.\nAirport districts.\nAssessment districts, benefit assessment districts, and sp

### 5. Train the Model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
training_args = Seq2SeqTrainingArguments(output_dir="./result", evaluation_strategy="epoch")

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:
# Save the model
model.save_pretrained('./model/')

### 6. Summerize Text

In [ ]:
from transformers import AutoModelForSeq2SeqLM, pipeline

model = AutoModelForSeq2SeqLM.from_pretrained('./model/')

text = "The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
summarizer(text, min_length=5, max_length=20)

[{'summary_text': '[unused2] [unused8] the inflation reduction act lowers prescription drug costs, health care [unused2] reduction act czech [unused2] [unused8]'}]